## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

### Radiology Entity Detection, Assertion Status Assignment, and Relation Extraction Pipeline

Overview:

The Radiology Entity Detection, Assertion Status Assignment, and Relation Extraction Pipeline is a comprehensive solution designed to enhance the analysis of radiology documents. Its primary focus lies in identifying crucial entities within radiology texts, assigning assertion statuses to these entities, and extracting relevant relations among them.


- **Model**: `en.explain_doc.clinical_radiology.pipeline`
- **Model Description**: A pipeline for detecting posology entities with the ner_radiology NER model, assigning their assertion status with assertion_dl_radiology model, and extracting relations between posology-related terminology with re_test_problem_finding relation extraction model.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "en-explain-doc-clinical-radiology-pipeline"

content_type = "application/json"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"

### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

---------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Perform real-time inference

In [6]:
import json
import pandas as pd
import os
import boto3


# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data_dicts):
    for data_dict in data_dicts:
        json_input_data = json.dumps(data_dict)    
        i = 1
        input_file_name = f'inputs/real-time/input{i}.json'
        output_file_name = f'outputs/real-time/out{i}.out'

        while os.path.exists(input_file_name) or os.path.exists(output_file_name):
            i += 1
            input_file_name = f'inputs/real-time/input{i}.json'
            output_file_name = f'outputs/real-time/out{i}.out'

        os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
        os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

        with open(input_file_name, 'w') as f:
            f.write(json_input_data)

        s3_client.put_object(Bucket=s3_bucket, Key=f"{model_name}/validation-input-json/real-time/{os.path.basename(input_file_name)}", Body=bytes(json_input_data.encode('UTF-8')))

        response = sm_runtime.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept="application/json",
            Body=json_input_data,
        )

        # Process response
        response_data = json.loads(response["Body"].read().decode("utf-8"))

        df = pd.DataFrame(response_data["predictions"])
        display(df)

        # Save response data to file
        with open(output_file_name, 'w') as f_out:
            json.dump(response_data, f_out, indent=4)


### Initial setup

In [7]:
docs = [
        """The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.""",

        """Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma."""
]

### Important Parameters 

- #### Output Levels

  The pipeline produces predictions at 3 distinct output levels:

  - document
  - chunk
  - relational

  Note: By default, the output level is chunk.

- #### Assertion

   DataType: `bool` 

  This parameter is only used with output levels document and chunk. It determines whether to include assertion status in the output.

### Chunk 

In the `chunk` output level, each document is repeated for every named entity or chunk recognized within the text, providing details such as the label, position, and confidence score for each entity.

**Input format**:
  
  
{
  "text": "Input text",
  "output_level": "chunk"
}


In [8]:
# Example usage:
data_dicts = [
    {
        "text": docs,
        "output_level": "chunk"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,ner_chunk,begin,end,ner_label,confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",insulin dependent diabetes,59,84,Disease_Syndrome_Disorder,0.5527
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",coronary artery disease,95,117,Disease_Syndrome_Disorder,0.5446667
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and assoc

### Chunk with Assertion

When the output level is set to `chunk` and assertion is set to `True`, it includes both the chunk output_level outputs and the assertion status. 

`Note`: Assertion status is determined through filtration based on specific columns `assertion_begin`, `assertion_end` from the assertion output and `begin`, `end` for chunk output level for each document.

**Input format**:
  
  
{
  "text": "Input text",
  "output_level": "chunk",
  "assertion": True
}


In [9]:
# Example usage:
data_dicts = [
    {
        "text": docs,
        "output_level": "chunk",
        "assertion": True
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,ner_chunk,begin,end,ner_label,confidence,assertion
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",insulin dependent diabetes,59,84,Disease_Syndrome_Disorder,0.5527,Confirmed
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",coronary artery disease,95,117,Disease_Syndrome_Disorder,0.5446667,Suspected
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left 

### Document

In the `document` output level, predictions are organized in a document-centric manner, providing a comprehensive overview of named entity recognition within each document. Each prediction aggregates all named entity chunks identified within a document, presenting the text along with its corresponding named entities, their positions, labels, and confidence scores.

**Input format**:
  
  
{
  "text": "Input text",
  "output_level": "document"
}


In [10]:
# Example usage:
data_dicts = [
    {
        "text": docs,
        "output_level": "document"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,ner_chunk,begin,end,ner_label,confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.","[insulin dependent diabetes, coronary artery disease, chronic renal insufficiency, peripheral vascular disease, diabetes, acute paraplegia, lower extremities, UTI, pressure-type wound, left foot, left great toe, large sacral, wound, laminectomy, diabetes, sacral, decubitus, left foot, pressure wound, diabetes, 5000, units, wounds, 40, units, 30, mg, 1, mg, 0.1, mg, 30, mg, 4, mg, 10, mg, 20, mg, 81, mg, 2, 400, mg, 5/325, mg, 1, 100, mg]","[59, 95, 120, 149, 196, 281, 299, 370, 444, 483, 497, 542, 555, 610, 812, 826, 833, 844, 854, 902, 933, 938, 978, 1010, 1013, 1056, 1059, 1087, 1089, 1113, 1117, 1141, 1144, 1162, 1164, 1182, 1185, 1203, 1206, 1224, 1227, 1243, 1276, 1280, 1305, 1311, 1357, 1422, 1426]","[84, 117, 146, 175, 203, 296, 315, 372, 462, 491, 510, 553, 559, 620, 819, 831, 841, 852, 867, 909, 936, 942, 983, 1011, 1017, 1057, 1060, 1087, 1090, 1115, 1118, 1142, 1145, 1162, 1165, 1183, 1186, 1204, 1207, 1225, 1228, 1243, 1278, 1281, 1309, 1312, 1357, 1424, 1427]","[Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, BodyPart, Disease_Syndrome_Disorder, Symptom, BodyPart, BodyPart, BodyPart, Symptom, Procedure, Disease_Syndrome_Disorder, BodyPart, Disease_Syndrome_Disorder, BodyPart, Symptom, Disease_Syndrome_Disorder, Measurements, Units, Symptom, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Measurements, Units, Measurements, Units, Measurements, Measurements, Units]","[0.5527, 0.5446667, 0.6552333, 0.72976667, 0.7741, 0.53215003, 0.68735, 0.9706, 0.6723, 0.66665, 0.5004333, 0.59064996, 0.851, 0.9424, 0.7412, 0.8043, 0.6002, 0.74415004, 0.43164998, 0.9535, 0.9881, 0.9977, 0.8557, 0.993, 0.9977, 0.9707, 0.9998, 0.9937, 0.9996, 0.9973, 0.9997, 0.9908, 0.9996, 0.9948, 0.9992, 0.9901, 0.9993, 0.9919, 0.9993, 0.9943, 0.9994, 0.9843, 0.9912, 0.9994, 0.9886, 0.9989, 0.9032, 0.9339, 0.9919]"
1,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.","[Bilateral bre

### Document with Assertion

When the output level is configured as `document` and the assertion parameter is set to `True`, it includes both the document output_level outputs and the assertion status (`assertion`, `assertion_begin`, and `assertion_end`).

**Input format**:
  
  
{
  "text": "Input text",
  "output_level": "document",
  "assertion": True
}


In [11]:
# Example usage:
data_dicts = [
    {
        "text": docs,
        "output_level": "document",
        "assertion": True
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,ner_chunk,begin,end,ner_label,confidence,assertion,assertion_begin,assertion_end
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.","[insulin dependent diabetes, coronary artery disease, chronic renal insufficiency, peripheral vascular disease, diabetes, acute paraplegia, lower extremities, UTI, pressure-type wound, left foot, left great toe, large sacral, wound, laminectomy, diabetes, sacral, decubitus, left foot, pressure wound, diabetes, 5000, units, wounds, 40, units, 30, mg, 1, mg, 0.1, mg, 30, mg, 4, mg, 10, mg, 20, mg, 81, mg, 2, 400, mg, 5/325, mg, 1, 100, mg]","[59, 95, 120, 149, 196, 281, 299, 370, 444, 483, 497, 542, 555, 610, 812, 826, 833, 844, 854, 902, 933, 938, 978, 1010, 1013, 1056, 1059, 1087, 1089, 1113, 1117, 1141, 1144, 1162, 1164, 1182, 1185, 1203, 1206, 1224, 1227, 1243, 1276, 1280, 1305, 1311, 1357, 1422, 1426]","[84, 117, 146, 175, 203, 296, 315, 372, 462, 491, 510, 553, 559, 620, 819, 831, 841, 852, 867, 909, 936, 942, 983, 1011, 1017, 1057, 1060, 1087, 1090, 1115, 1118, 1142, 1145, 1162, 1165, 1183, 1186, 1204, 1207, 1225, 1228, 1243, 1278, 1281, 1309, 1312, 1357, 1424, 1427]","[Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, BodyPart, Disease_Syndrome_Disorder, Symptom, BodyPart, BodyPart, BodyPart, Symptom, Procedure, Disease_Syndrome_Disorder, BodyPart, Disease_Syndrome_Disorder, BodyPart, Symptom, Disease_Syndrome_Disorder, Measurements, Units, Symptom, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Measurements, Units, Measurements, Units, Measurements, Measurements, Units]","[0.5527, 0.5446667, 0.6552333, 0.72976667, 0.7741, 0.53215003, 0.68735, 0.9706, 0.6723, 0.66665, 0.5004333, 0.59064996, 0.851, 0.9424, 0.7412, 0.8043, 0.6002, 0.74415004, 0.43164998, 0.9535, 0.9881, 0.9977, 0.8557, 0.993, 0.9977, 0.9707, 0.9998, 0.9937, 0.9996, 0.9973, 0.9997, 0.9908, 0.9996, 0.9948, 0.9992, 0.9901, 0.9993, 0.9919, 0.9993, 0.9943, 0.9994, 0.9843, 0.9912, 0.9994, 0.9886, 0.9989, 0.9032, 0.9339, 0.9919]","[Confirmed, Suspected, Negative, Negative, Suspected, Suspected, Suspected, Confirmed, Confirmed, Suspected, Confirmed, Confirmed, Confirmed, Negative]","[59, 95, 120, 149, 196, 281, 370, 444, 548, 812, 826, 849, 902, 978]","[84, 117, 146, 175, 203, 296, 372, 462, 559, 819, 841, 867, 909, 983]"
1,"Bilateral breast ultrasound was subsequently per

### Relational

The `relational` output level involves identifying and specifying relationships between named entities or chunks within the text, often accompanied by confidence scores indicating the model's certainty in predicting those relationships.

**Input format**:
  
  
{
  "text": "Input text",
  "output_level": "relation"
}


In [12]:
# Example usage:
data_dicts = [
    {
        "text": docs,
        "output_level": "relational"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,relation,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.","[is_related, is_related]","[pressure-type wound, pressure-type wound]","[444, 444]","[462, 462]","[Symptom, Symptom]","[left foot, left great toe]","[483, 497]","[491, 510]","[BodyPart, BodyPart]","[0.999964, 0.9907776]"
1,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.","[is_related, is_related, is_related, is_related, is_related, is_related]","[Bilateral breast, Bilateral breast, ultrasound, ovoid mass, mass, isoechoic echotexture]","[0, 0, 17, 78, 210, 228]","[15, 15, 26, 87, 213, 248]","[BodyPart, BodyPart, Imaging_Test, ImagingFindings, ImagingFindings, ImagingFindings]","[ultrasound, ovoid mass, ovoid mass, 0.5 x 0.5 x 0.4 cm, adjacent muscle, adjacent muscle]","[17, 78, 78, 113, 257, 257]","[26, 87, 87, 130, 271, 271]","[Imaging_Test, ImagingFindings, ImagingFindings, Measurements, BodyPart, BodyPart]","[1.0, 0.99999714, 0.99956936, 1.0, 0.9976392, 0.9999993]"


### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [13]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [15]:
import os

validation_file_name_1 = "input_1.json"
validation_file_name_2 = "input_2.json"
validation_file_name_3 = "input_3.json"
validation_file_name_4 = "input_4.json"
validation_file_name_5 = "input_5.json"

validation_input_path = f"s3://{s3_bucket}/{model_name}/validation-input-json/batch"
validation_output_path = f"s3://{s3_bucket}/{model_name}/validation-output-json/batch"

input_dir = 'inputs/batch'
output_dir = 'outputs/batch'

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [16]:
import json

def write_and_upload_to_s3(json_input_data, file_name):

    json_data = json.dumps(json_input_data)

    with open(file_name, "w") as f:
        f.write(json_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input-json/batch/{os.path.basename(file_name)}",
        Body=(bytes(json_data.encode("UTF-8"))),
    )


In [17]:
# Define input JSON data for each validation file
input_json_data = {
    validation_file_name_1: {
        "text": docs,
        "output_level": "chunk"
    },
    validation_file_name_2: {
        "text": docs,
        "output_level": "chunk",
        "assertion": True
    },
    validation_file_name_3: {
        "text": docs,
        "output_level": "document",
    },
    validation_file_name_4: {
        "text": docs,
        "output_level": "document",
        "assertion": True
    },
    validation_file_name_5: {
        "text": docs,
        "output_level": "relational",
    }
}

# Write and upload each input JSON data to S3
for file_name, json_data in input_json_data.items():
    write_and_upload_to_s3(json_data, f"{input_dir}/{file_name}")


In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    accept="application/json",
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_output_and_save(validation_file_name, output_file_name):

    output_file_path = f"{output_dir}/{output_file_name}"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data["predictions"])
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)



### Chunk 

In [20]:
process_s3_output_and_save(validation_file_name_1, "out_1.out")

,document,ner_chunk,begin,end,ner_label,confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",insulin dependent diabetes,59,84,Disease_Syndrome_Disorder,0.5527
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",coronary artery disease,95,117,Disease_Syndrome_Disorder,0.5446667
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and assoc

### Chunk with Assertion

In [21]:
process_s3_output_and_save(validation_file_name_2, "out_2.out")

,document,ner_chunk,begin,end,ner_label,confidence,assertion
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",insulin dependent diabetes,59,84,Disease_Syndrome_Disorder,0.5527,Confirmed
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",coronary artery disease,95,117,Disease_Syndrome_Disorder,0.5446667,Suspected
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left 

### Document

In [22]:
process_s3_output_and_save(validation_file_name_3, "out_3.out")

,document,ner_chunk,begin,end,ner_label,confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.","[insulin dependent diabetes, coronary artery disease, chronic renal insufficiency, peripheral vascular disease, diabetes, acute paraplegia, lower extremities, UTI, pressure-type wound, left foot, left great toe, large sacral, wound, laminectomy, diabetes, sacral, decubitus, left foot, pressure wound, diabetes, 5000, units, wounds, 40, units, 30, mg, 1, mg, 0.1, mg, 30, mg, 4, mg, 10, mg, 20, mg, 81, mg, 2, 400, mg, 5/325, mg, 1, 100, mg]","[59, 95, 120, 149, 196, 281, 299, 370, 444, 483, 497, 542, 555, 610, 812, 826, 833, 844, 854, 902, 933, 938, 978, 1010, 1013, 1056, 1059, 1087, 1089, 1113, 1117, 1141, 1144, 1162, 1164, 1182, 1185, 1203, 1206, 1224, 1227, 1243, 1276, 1280, 1305, 1311, 1357, 1422, 1426]","[84, 117, 146, 175, 203, 296, 315, 372, 462, 491, 510, 553, 559, 620, 819, 831, 841, 852, 867, 909, 936, 942, 983, 1011, 1017, 1057, 1060, 1087, 1090, 1115, 1118, 1142, 1145, 1162, 1165, 1183, 1186, 1204, 1207, 1225, 1228, 1243, 1278, 1281, 1309, 1312, 1357, 1424, 1427]","[Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, BodyPart, Disease_Syndrome_Disorder, Symptom, BodyPart, BodyPart, BodyPart, Symptom, Procedure, Disease_Syndrome_Disorder, BodyPart, Disease_Syndrome_Disorder, BodyPart, Symptom, Disease_Syndrome_Disorder, Measurements, Units, Symptom, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Measurements, Units, Measurements, Units, Measurements, Measurements, Units]","[0.5527, 0.5446667, 0.6552333, 0.72976667, 0.7741, 0.53215003, 0.68735, 0.9706, 0.6723, 0.66665, 0.5004333, 0.59064996, 0.851, 0.9424, 0.7412, 0.8043, 0.6002, 0.74415004, 0.43164998, 0.9535, 0.9881, 0.9977, 0.8557, 0.993, 0.9977, 0.9707, 0.9998, 0.9937, 0.9996, 0.9973, 0.9997, 0.9908, 0.9996, 0.9948, 0.9992, 0.9901, 0.9993, 0.9919, 0.9993, 0.9943, 0.9994, 0.9843, 0.9912, 0.9994, 0.9886, 0.9989, 0.9032, 0.9339, 0.9919]"
1,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.","[Bilateral bre

### Document with Assertion

In [23]:
process_s3_output_and_save(validation_file_name_4, "out_4.out")

,document,ner_chunk,begin,end,ner_label,confidence,assertion,assertion_begin,assertion_end
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.","[insulin dependent diabetes, coronary artery disease, chronic renal insufficiency, peripheral vascular disease, diabetes, acute paraplegia, lower extremities, UTI, pressure-type wound, left foot, left great toe, large sacral, wound, laminectomy, diabetes, sacral, decubitus, left foot, pressure wound, diabetes, 5000, units, wounds, 40, units, 30, mg, 1, mg, 0.1, mg, 30, mg, 4, mg, 10, mg, 20, mg, 81, mg, 2, 400, mg, 5/325, mg, 1, 100, mg]","[59, 95, 120, 149, 196, 281, 299, 370, 444, 483, 497, 542, 555, 610, 812, 826, 833, 844, 854, 902, 933, 938, 978, 1010, 1013, 1056, 1059, 1087, 1089, 1113, 1117, 1141, 1144, 1162, 1164, 1182, 1185, 1203, 1206, 1224, 1227, 1243, 1276, 1280, 1305, 1311, 1357, 1422, 1426]","[84, 117, 146, 175, 203, 296, 315, 372, 462, 491, 510, 553, 559, 620, 819, 831, 841, 852, 867, 909, 936, 942, 983, 1011, 1017, 1057, 1060, 1087, 1090, 1115, 1118, 1142, 1145, 1162, 1165, 1183, 1186, 1204, 1207, 1225, 1228, 1243, 1278, 1281, 1309, 1312, 1357, 1424, 1427]","[Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, Disease_Syndrome_Disorder, BodyPart, Disease_Syndrome_Disorder, Symptom, BodyPart, BodyPart, BodyPart, Symptom, Procedure, Disease_Syndrome_Disorder, BodyPart, Disease_Syndrome_Disorder, BodyPart, Symptom, Disease_Syndrome_Disorder, Measurements, Units, Symptom, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Units, Measurements, Measurements, Units, Measurements, Units, Measurements, Measurements, Units]","[0.5527, 0.5446667, 0.6552333, 0.72976667, 0.7741, 0.53215003, 0.68735, 0.9706, 0.6723, 0.66665, 0.5004333, 0.59064996, 0.851, 0.9424, 0.7412, 0.8043, 0.6002, 0.74415004, 0.43164998, 0.9535, 0.9881, 0.9977, 0.8557, 0.993, 0.9977, 0.9707, 0.9998, 0.9937, 0.9996, 0.9973, 0.9997, 0.9908, 0.9996, 0.9948, 0.9992, 0.9901, 0.9993, 0.9919, 0.9993, 0.9943, 0.9994, 0.9843, 0.9912, 0.9994, 0.9886, 0.9989, 0.9032, 0.9339, 0.9919]","[Confirmed, Suspected, Negative, Negative, Suspected, Suspected, Suspected, Confirmed, Confirmed, Suspected, Confirmed, Confirmed, Confirmed, Negative]","[59, 95, 120, 149, 196, 281, 370, 444, 548, 812, 826, 849, 902, 978]","[84, 117, 146, 175, 203, 296, 372, 462, 559, 819, 841, 867, 909, 983]"
1,"Bilateral breast ultrasound was subsequently per

### Relational

In [24]:
process_s3_output_and_save(validation_file_name_5, "out_5.out")

,document,relation,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.","[is_related, is_related]","[pressure-type wound, pressure-type wound]","[444, 444]","[462, 462]","[Symptom, Symptom]","[left foot, left great toe]","[483, 497]","[491, 510]","[BodyPart, BodyPart]","[0.999964, 0.9907776]"
1,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.","[is_related, is_related, is_related, is_related, is_related, is_related]","[Bilateral breast, Bilateral breast, ultrasound, ovoid mass, mass, isoechoic echotexture]","[0, 0, 17, 78, 210, 228]","[15, 15, 26, 87, 213, 248]","[BodyPart, BodyPart, Imaging_Test, ImagingFindings, ImagingFindings, ImagingFindings]","[ultrasound, ovoid mass, ovoid mass, 0.5 x 0.5 x 0.4 cm, adjacent muscle, adjacent muscle]","[17, 78, 78, 113, 257, 257]","[26, 87, 87, 130, 271, 271]","[Imaging_Test, ImagingFindings, ImagingFindings, Measurements, BodyPart, BodyPart]","[1.0, 0.99999714, 0.99956936, 1.0, 0.9976392, 0.9999993]"


In [25]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-explain-doc-clinical-radiology-pipel-2024-03-03-08-14-37-926


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

